In [ ]:
import requests
from bs4 import BeautifulSoup as soup
from urllib.parse import urlencode
import contextlib
import datetime
from urllib.request import urlopen
from pushover import Pushover

In [ ]:
class ForbiddenPlanet:
    
    
    def __init__(self, get_live_sitemap = True):
        
        self.sitemap_home = 'https://forbiddenplanet.com/sitemap.xml'
        self.webpage_home = 'https://forbiddenplanet.com'
        if get_live_sitemap:
            self.products = self.get_current_products()
            
        else:
            print('dam')
    
    def get_max_product_pages(self):

        res = requests.get(self.sitemap_home)
        data = res.text
        sres = soup(data)
        relivant_pages = [x.text.strip() for x in sres.findAll('loc') if '/sitemap-products' in x.text]
        max_page = max([int(x.split('?p=')[-1].strip()) for x in relivant_pages if '?p=' in x])

        return max_page
        
    def get_current_products(self):
    
        current_max_pages = self.get_max_product_pages()
        products = []
        for page in range(1,current_max_pages+1):
            print(f'Currently parsing page {page} of {current_max_pages}...')
            res = requests.get(f'https://forbiddenplanet.com/sitemap-products.xml?p={page}').text
            sres = soup(res)
            products.append([x.loc.text.strip() for x in sres.findAll('url')])

        flattened_products = [i for j in products for i in j] 
        return flattened_products  
        
    def get_stock_count(self, product_url):
        
        res = soup(requests.get(product_url).text)
        stock_number = [x.string.strip().split('stock_count":')[-1].split(',')[0] for x in res.find_all('script') if x.string if 'stock_count' in x.string]
        stock_number, = [float('nan') if not x.isdigit() else int(x) for x in stock_number]
        return stock_number
    
    def hyphen_wrapper(self, text):
        if len(text.split()) == 1 and '*' in text:
            return text.replace('*','')
        else:
            return '-'+'-'.join(text.split())
    
    
    def set_comic_scope(self, series, issue_number = None, artist = None, cover=None, variant = False, single_issue = True):
    
        match_strings = []
        match_strings.append(self.hyphen_wrapper(series))
        if single_issue and issue_number:
            match_strings.append(self.hyphen_wrapper(str(issue_number)))
        if artist:
            match_strings.append(self.hyphen_wrapper(artist))    
        if cover:
            match_strings.append(self.hyphen_wrapper(f'cover-{cover}'))    
        if variant:
            match_strings.append(self.hyphen_wrapper('variant'))

        return match_strings
    
    
    def exercute_search_vectors(self, comic_scope, search_vector = 'sitemap'):
        
        if search_vector == 'sitemap':
            current_products = self.products
            matching_comics = [url for url in current_products if all(feature in url for feature in comic_scope)]
            return matching_comics


In [ ]:
fp = ForbiddenPlanet(get_live_sitemap = True)

In [ ]:
comic_scope = fp.set_comic_scope(series = 'batman', issue_number = 120, artist = 'jock', cover = False, variant = True)
comic_scope = fp.set_comic_scope(series = 'blade runner 2029', issue_number = 1, artist = 'momoko', cover =False, variant = True)

In [ ]:
fp.exercute_search_vectors(comic_scope)

In [ ]:
fp.get_stock_count('https://forbiddenplanet.com/358139-batman-124-cover-d-filya-bratukhin-card-stock-variant/')

In [ ]:
fp.exercute_search_vectors(comic_scope)

In [ ]:
import fpcomics as fp

In [ ]:
from fpcomics import PushOver

In [ ]:
push = PushOver()

In [ ]:
stock_test_comic_1 = {'Batman 125 1:125':'https://forbiddenplanet.com/360496-batman-125-cover-i-alex-garner-foil-variant/'}
stock_test_comic_2 = {'Batman 125 1:250':'https://forbiddenplanet.com/360497-batman-125-cover-j-chip-zdarsky-card-stock-variant/'}
stock_test_comic_3 = {'Batman 125 1:500':'https://forbiddenplanet.com/360498-batman-125-cover-k-jim-lee-scott-williams-black-white-card-stock-variant/'}

live_test_comic_4 = {'Batman 127 Jock Foil':['-batman', '-127', '-jock', '-variant']}
live_test_comic_5 = {'Batman Beyond #1 Cover A':['-batman-beyond', '-1', '-murphy', '-cover-a']}

In [ ]:
stock_list = [stock_test_comic_1, stock_test_comic_2, stock_test_comic_3]

In [ ]:
#a = push.set_comic_scope(series = 'batman', issue_number = 127, artist = 'jock', cover = False, variant = True)
#b = push.set_comic_scope(series = 'batman beyond', issue_number = 1, artist = 'murphy', cover = 'a', variant = False)

In [ ]:
live_list = [['-batman', '-127', '-jock', '-variant'], ['-batman-beyond', '-1', '-murphy', '-cover-a']]

In [ ]:
#notif = push.format_live_notification(live_list)
#notif = push.format_oos_notification(stock_list)
notif = push.format_watchlist_notification(stock_list)

In [ ]:
push.send_notification(notif)

In [ ]:
def watchlist_main():
    push = PushOver()
    stock_test_comic_1 = {'Batman 125 1:125':'https://forbiddenplanet.com/360496-batman-125-cover-i-alex-garner-foil-variant/'}
    stock_test_comic_2 = {'Batman 125 1:250':'https://forbiddenplanet.com/360497-batman-125-cover-j-chip-zdarsky-card-stock-variant/'}
    stock_test_comic_3 = {'Batman 125 1:500':'https://forbiddenplanet.com/360498-batman-125-cover-k-jim-lee-scott-williams-black-white-card-stock-variant/'}
    stock_list = [stock_test_comic_1, stock_test_comic_2, stock_test_comic_3]
    notif = push.format_watchlist_notification(stock_list)
    push.send_notification(notif)
    

In [ ]:
stock_list = [('Batman 125 1:125','https://forbiddenplanet.com/360496-batman-125-cover-i-alex-garner-foil-variant/'),('Batman 125 1:250','https://forbiddenplanet.com/360497-batman-125-cover-j-chip-zdarsky-card-stock-variant/'),('Batman 125 1:500','https://forbiddenplanet.com/360498-batman-125-cover-k-jim-lee-scott-williams-black-white-card-stock-variant/')]

In [ ]:
bjson = build_new_watchlist('Batman List', stock_list)

In [ ]:
def build_new_watchlist(listname, entries):
    return {listname : dict(entries)}
    
    

In [ ]:
ex_comics = {'Batman 124 Bratukhin Variant':'https://forbiddenplanet.com/358139-batman-124-cover-d-filya-bratukhin-card-stock-variant/','Batman 125 Zdarsky':'https://forbiddenplanet.com/360497-batman-125-cover-j-chip-zdarsky-card-stock-variant/', 'Batman: Beyond The White Knight #1': 'https://forbiddenplanet.com/355848-batman-beyond-the-white-knight-1-2nd-printing-foil-variant/?utm_medium=promotion&utm_source=recently-viewed&utm_campaign=recently-viewed'}

In [ ]:
with open('comic_lists/watchlist.json', 'w') as fp:
    json.dump(bjson , fp)

In [ ]:
def load_json_file(list_type = 'watchlist'):
    
    f = open(f'comic_lists/{list_type}.json')
    data = json.load(f)
    f.close()
    return data
    

In [ ]:
import json
  
# Opening JSON file
f = open('comic_lists/watchlist.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
f.close()

In [ ]:
import http.client, urllib
conn = http.client.HTTPSConnection("api.pushover.net:443")
conn.request("POST", "/1/messages.json",
  urllib.parse.urlencode({
    "token": "as7247rm3nynymrm5giszfbcj7u81z",
    "user": "urvyua61kboivihbpw7idicikze13z",
    "message": "hello world <b>word</b>",
    "html":1,  
  }), { "Content-type": "application/x-www-form-urlencoded" })
conn.getresponse()

In [ ]:
make_tiny('https://forbiddenplanet.com/355848-batman-beyond-the-white-knight-1-2nd-printing-foil-variant/?utm_medium=promotion&utm_source=recently-viewed&utm_campaign=recently-viewed')

In [ ]:
from urllib.parse import urlencode
import contextlib
import datetime
from urllib.request import urlopen

In [ ]:
class PushOver:
    
    def __init__(self):
        
        api_key = "as7247rm3nynymrm5giszfbcj7u81z"
        user_key = "urvyua61kboivihbpw7idicikze13z"
        self.po = Pushover(api_key)
        self.po.user(user_key)
        
        
    def send_notification(self, notification: str, notification_type: str = 'watchlist') -> None:
        
        msg = self.po.msg(notification)
        titles = {'watchlist':"WATCHLIST STOCK CHECK", 'ooslist':"OOS STOCK ALERT", 'livelist': "LIVE TITLE ALERT"}
        msg.set("title", titles[notification_type])
        msg.set("html",1)
        self.po.send(msg)
            
        
    def format_watchlist_notification(self, comic_titles: dict) -> str:
        message=[]
        message.append(f"==============================")
        message.append(f'<b>{datetime.datetime.now().strftime("%d-%B-%Y %H:%M:%S")}</b>')
        message.append(f"==============================")
        for comic_title in ex_comics.items():

            stock = fp.get_stock_count(comic_title[-1])
            tiny_url = make_tiny(comic_title[-1])
            if stock>0:
                message.append(f"<font color='yellow'>{comic_title[0]}</font> \n Currently <font color='green'>IN</font> stock! :) with a Qty: <b>{stock}</b> \n <a href='{tiny_url}'>Buy now!</a>")
                message.append(f"----------------------------------")
            else:
                message.append(f"<font color='yellow'>{comic_title[0]}</font> \n Currently <font color='red'>NOT</font> in stock! :( ")
                message.append(f"----------------------------------")

        message = '\n'.join(message)          
        return message    
        
        
    def format_oos_notification(self, comic_titles: dict) -> str:
    
        message=[]
        message.append(f"==============================")
        message.append(f'<b>{datetime.datetime.now().strftime("%d-%B-%Y %H:%M:%S")}</b>')
        message.append(f"==============================")
        for comic_title in comic_titles.items():

            stock = fp.get_stock_count(comic_title[-1])
            tiny_url = make_tiny(comic_title[-1])
            if stock>0:
                message.append(f"<font color='yellow'>{comic_title[0]}</font> \n Looks to be back <font color='green'>IN</font> stock! :) with a Qty: <b>{stock}</b> \n <a href='{tiny_url}'>Buy now!</a>")
                message.append(f"----------------------------------")
            else:
                continue

        message = '\n'.join(message) 
    return message   


    def format_live_notification(self, comic_titles: dict) -> str:

        message=[]
        message.append(f"==============================")
        message.append(f'<b>{datetime.datetime.now().strftime("%d-%B-%Y %H:%M:%S")}</b>')
        message.append(f"==============================")

        for comic_scope in comic_titles.items():
            print(comic_scope)
            live_status = fp.exercute_search_vectors(comic_scope[-1])
            print('*************')
            print(live_status, 'status')

            if len(live_status) == 1:
                stock = fp.get_stock_count(live_status[0])
                tiny_url = make_tiny(live_status[0])
                if stock > 0:
                    message.append(f"<font color='yellow'>{scope_format_convertion(comic_scope[-1])}</font> \n is <font color='green'>LIVE</font> and <font color='green'>IN STOCK</font> Qty: <b>{stock}</b> \n <a href='{tiny_url}'>Buy now!</a>")
                    message.append(f"----------------------------------")
                elif stock == 0:
                    message.append(f"<font color='yellow'>{scope_format_convertion(comic_scope[-1])}</font> \n is <font color='green'>LIVE</font> but <font color='red'>OOS</font> \n <a href='{tiny_url}'>Watch here!</a>")
                    message.append(f"----------------------------------")
            elif not live_status:
                print('YEAHHH')
                message.append(f"<font color='yellow'>{scope_format_convertion(comic_scope[-1])}</font> \n is <font color='red'>NOT</font> live yet. <b>{stock}</b>")
                message.append(f"----------------------------------")


        message = '\n'.join(message) 
        return message
    
    
    def scope_format_convertion(comic_scope: list) -> str:
        comic_scope = [x.replace('-',' ').title() for x in comic_scope]
        return ' -'.join(comic_scope).strip()
    
    def make_tiny(url: str) -> str:
        request_url = ('http://tinyurl.com/api-create.php?' + urlencode({'url':url}))   
        with contextlib.closing(urlopen(request_url)) as response:                      
            return response.read().decode('utf-8 ') 
        
        
        
        

In [ ]:
def check_upcoming_list():
    
    for comic_title in upcoming_list:
        

In [ ]:
ex_comics = {'Batman 124 Bratukhin Variant':'https://forbiddenplanet.com/358139-batman-124-cover-d-filya-bratukhin-card-stock-variant/','Batman 125 Zdarsky':'https://forbiddenplanet.com/360497-batman-125-cover-j-chip-zdarsky-card-stock-variant/', 'Batman: Beyond The White Knight #1': 'https://forbiddenplanet.com/355848-batman-beyond-the-white-knight-1-2nd-printing-foil-variant/?utm_medium=promotion&utm_source=recently-viewed&utm_campaign=recently-viewed'}

In [ ]:
message = format_oos_notification()

In [ ]:
send_notification(message, notification_type = 'ooslist')

In [ ]:
#fp.set_comic_scope(series = 'batman', issue_number = 125, artist = 'jock', cover =False, variant = True)
comic_scope_1 = fp.set_comic_scope(series = 'batman', issue_number = 125, artist = 'jock', cover =False, variant = True)
comic_scope_2 = fp.set_comic_scope(series = 'batman', issue_number = 128, artist = 'jock', cover =False, variant = True)


In [ ]:
ex_comics = {'Batman 125 Jock': comic_scope_1, 'Batman 128 Jock' : comic_scope_2}

In [ ]:
message = format_live_notification()

In [ ]:
def scope_format_convertion(comic_scope):
    comic_scope = [x.replace('-',' ').title() for x in comic_scope]
    return ' -'.join(comic_scope).strip()

In [ ]:
scope_format_convertion(comic_scope)

In [ ]:
send_notification(message, notification_type = 'livelist')

In [ ]:
list(comics.values())

In [ ]:
comics = load_json_file('watchlist')

In [1]:
from fpcomics import PushOver

In [2]:
def watchlist_main(watchlist):
    push = PushOver(get_live_sitemap = False)
    comics = load_json_file(watchlist)
    notif = push.format_watchlist_notification(comics)
    push.send_notification(notif, *list(comics.keys()))

In [5]:
watchlist_main('watchlist')

/Users/matt/Documents/fp_code/fpcomics.py:52: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 52 of the file /Users/matt/Documents/fp_code/fpcomics.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  res = soup(requests.get(product_url).text)


In [3]:
def load_json_file(list_type = 'watchlist'):
    
    f = open(f'comic_lists/{list_type}.json')
    data = json.load(f)
    f.close()
    return data

In [4]:
import json